# 트리 기반의 회귀 모델
***

이번 챕터에서는 트리 기반의 회귀 모델에 대해서 알아보도록 하겠습니다. 일반적으로 선형 회귀 모델은 최적의 회귀 계수를 찾아내는 것이 목표인데 반해 트리 기반 모델은 회귀를 위한 트리를 만들어 예측을 하게 됩니다. 트리 기반의 분류 모델은 클래스를 결정하지만 회귀 모델은 노드에 속한 데이터들의 평균값을 구하여 예측을 합니다. 

DecisionTree, RandomForest, Lightgbm, XGBoost, GradientBoosting 등 트리 기반의 알고리즘들은 기본적으로 **CART(Classification And Regression Tree)알고리즘**을 기반으로 하기 때문에 회귀도 가능합니다. 앞서 공부한 선형 회귀 모델들과 트리 기반의 회귀 모델들의 성능을 비교해보도록 하겠습니다. 

In [15]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, \
HistGradientBoostingRegressor, RandomForestRegressor,VotingRegressor, StackingRegressor
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression
from sklearn.datasets import load_boston
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

In [8]:
X = pd.DataFrame(load_boston().data, columns = load_boston().feature_names)
y = load_boston().target

In [10]:
X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


총 13개의 모델들을 10FOLD cross validation으로 RMSE를 평가해보도록 하겠습니다.

In [16]:
lr = LinearRegression()
ridge = Ridge(random_state = 110)
lasso = Lasso(random_state = 110)
ela = ElasticNet(random_state = 110)
dt = DecisionTreeRegressor(random_state = 110)
rf = RandomForestRegressor(random_state = 110)
etr = ExtraTreesRegressor(random_state = 110)
gb = GradientBoostingRegressor(random_state = 110)
abr = AdaBoostRegressor(random_state = 110)
bgr = BaggingRegressor(random_state = 110)
hgb = HistGradientBoostingRegressor(random_state = 110)
lgb = LGBMRegressor(random_state = 110)
xgb = XGBRegressor(random_state = 110)

In [17]:
models = [lr, ridge, lasso, ela, dt, rf, etr, gb, abr, bgr, hgb, lgb, xgb]

for문을 활용해서 모든 모델을 학습해보도록 하겠습니다.

In [31]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 110)
rmse_list = []
for model in models :
    cv_mse = cross_val_score(model, X, y, cv = kf, scoring = 'neg_mean_squared_error')
    rmse = round(np.mean(np.sqrt(-cv_mse)), 5)
    rmse_list.append(rmse)
    print(f'{model.__class__.__name__}의 10FOLD RMSE는 {rmse}')

LinearRegression의 10FOLD RMSE는 4.78593
Ridge의 10FOLD RMSE는 4.7794
Lasso의 10FOLD RMSE는 5.21638
ElasticNet의 10FOLD RMSE는 5.15703
DecisionTreeRegressor의 10FOLD RMSE는 4.90637
RandomForestRegressor의 10FOLD RMSE는 3.16302
ExtraTreesRegressor의 10FOLD RMSE는 2.97028
GradientBoostingRegressor의 10FOLD RMSE는 3.05332
AdaBoostRegressor의 10FOLD RMSE는 3.81828
BaggingRegressor의 10FOLD RMSE는 3.32197
HistGradientBoostingRegressor의 10FOLD RMSE는 3.23143
LGBMRegressor의 10FOLD RMSE는 3.12634
XGBRegressor의 10FOLD RMSE는 3.15278


모델명과 RMSE를 데이터 프레임으로 매핑시켜보겠습니다.

In [30]:
result = pd.DataFrame({'model' : [x.__class__.__name__ for x in models], 'RMSE' : rmse_list})
result

,model,RMSE
0,LinearRegression,4.78593
1,Ridge,4.77940
2,Lasso,5.21638
3,ElasticNet,5.15703
4,DecisionTreeRegressor,4.90637
5,RandomForestRegressor,3.16302
6,ExtraTreesRegressor,2.97028
7,GradientBoostingRegressor,3.05332
8,AdaBoostRegressor,3.81828
9,BaggingRegressor,3.32197


In [35]:
result.sort_values(by = 'RMSE')

,model,RMSE
6,ExtraTreesRegressor,2.97028
7,GradientBoostingRegressor,3.05332
11,LGBMRegressor,3.12634
12,XGBRegressor,3.15278
5,RandomForestRegressor,3.16302
10,HistGradientBoostingRegressor,3.23143
9,BaggingRegressor,3.32197
8,AdaBoostRegressor,3.81828
1,Ridge,4.77940
0,LinearRegression,4.78593


RMSE를 기준으로 오름차순한 결과 선형 회귀 모델은 최하위의 성능을 기록했고 트리 기반의 회귀 모델이 성능이 우수한 것을 알 수 있었습니다. 

상위 3개의 모델들로 stacking과 voting을 진행해 결과를 보도록 하겠습니다.

In [38]:
stc = StackingRegressor(estimators = [('etr', etr), ('gb', gb), ('lgb', lgb)], final_estimator = xgb)

In [42]:
cv_mse = cross_val_score(stc, X, y, cv = kf, scoring = 'neg_mean_squared_error')

In [46]:
print(f'{stc.__class__.__name__}의 10FOLD RMSE는 {np.mean(np.sqrt(-cv_mse))}')

StackingRegressor의 10FOLD RMSE는 4.343802205061216


In [50]:
vot = VotingRegressor(estimators = [('etr', etr), ('gb', gb), ('lgb', lgb)], weights = [.5, .3, .2])

In [51]:
cv_mse = cross_val_score(vot, X, y, cv = kf, scoring = 'neg_mean_squared_error')

In [52]:
print(f'{vot.__class__.__name__}의 10FOLD RMSE는 {np.mean(np.sqrt(-cv_mse))}')

VotingRegressor의 10FOLD RMSE는 2.9008439310480645


Stacking의 경우 RMSE는 하위권에 속할 만큼 성능이 좋지 못했지만 Voting의 경우는 최고의 성능을 기록했습니다. 지금까지 회귀의 기본 개념부터 선형 회귀 모델, 트리기반 회귀 모델들을 공부해보았습니다. 다음 챕터에서는 kaggle의 open competition인 'House Price' 데이터를 사용해 실제 submission 제출까지 해보도록 하겠습니다. 감사합니다.